In [ ]:
# 필수 라이브러리 설치
!pip install roboflow singleton-decorator

In [ ]:
import os
import sys
import cv2
import numpy as np
import torch
from singleton_decorator import singleton

In [ ]:
# YOLOv5 레포지토리 클론 및 이동
if not os.path.exists('/content/yolov5'):
    !git clone https://github.com/ultralytics/yolov5 /content/yolov5
%cd /content/yolov5
!pip install -r requirements.txt

/content/yolov5


In [ ]:
# Roboflow에서 데이터셋 다운로드
from roboflow import Roboflow
rf = Roboflow(api_key="gUNvjxIQ9rKYruOxpb3t")
project = rf.workspace("forklift-foujc").project("forklift-labeling")
version = project.version(4)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to forklift-labeling-4 in yolov5pytorch:: 100%|██████████| 17688/17688 [00:05<00:00, 3135.96it/s]


In [ ]:
@singleton
class ClsYoloGPU:
    def __init__(self, Param):
        self.Param = Param
        self.Param.LoadDbValue()
        self.__initCls()
        self.__deviceInfo()

    def __initCls(self):
        try:
            ModelFile = self.Param.model
            framework = os.path.join(os.getcwd(), 'yolov5')
            self.__model = torch.hub.load(framework, 'custom', path=ModelFile, source='local')
            self.__model.conf = self.Param.confThresh
            self.__model.iou = self.Param.iouThresh
        except Exception as Ex:
            coName = sys._getframe().f_code.co_name
            print(f'{coName} = {Ex}')

    def __deviceInfo(self):
        device = (
            "cuda"
            if torch.cuda.is_available()
            else "mps"
            if torch.backends.mps.is_available()
            else "cpu"
        )
        print(f"Using {device} device")
        if device == "cuda":
            print(torch.cuda.get_device_name(0))
            print(torch.cuda.current_device())
            print(torch.cuda.device_count())

    def detectChar(self, img: np.ndarray) -> list:
        LsResult = []  # 초기화
        try:
            (h, w) = img.shape[:2]
            npCvs = np.copy(img)
            e1 = cv2.getTickCount()

            result = self.__model([img], size=1024)
            e2 = cv2.getTickCount()
            t = (e2 - e1) / cv2.getTickFrequency()
            print(f'Work Time = {t}sec')

            result.print()
            print(self.__model.conf)
            print(self.__model.iou)
            print(result.xyxy[0])
            print(result.pandas().xyxy[0])

            box = result.xyxy[0]
            print(f'yolov5 box = type {type(result.xyxy[0])}')

            resnp = result.xyxy[0].cpu().numpy()
            reInt = resnp.astype(np.uint64)

            LsOrg = resnp.tolist()
            LsResult = []

            for i, elem in enumerate(reInt.tolist()):
                TxDis = ''
                Score = LsOrg[i][4]
                tpLT = (elem[0], elem[1])
                tpRb = (elem[2], elem[3])
                TxDis = '{0}'.format(elem[5])

                tpData = (TxDis, Score, tpLT, tpRb)
                LsResult.append(tpData)

        except Exception as Ex:
            coName = sys._getframe().f_code.co_name
            print(f'{coName} = {Ex}')

        return LsResult

In [ ]:
# ClsRecipe 클래스는 사용자 정의 클래스입니다. 이 예제에서는 더미 클래스로 대체합니다.
class ClsRecipe:
    def __init__(self, model, confThresh, iouThresh):
        self.model = model
        self.confThresh = confThresh
        self.iouThresh = iouThresh

    def LoadDbValue(self):
        pass  # 데이터베이스에서 값을 로드하는 로직을 추가합니다.

In [ ]:
# 예제 파라미터 설정
param = ClsRecipe(model='yolov5s.pt', confThresh=0.25, iouThresh=0.45)
yolo = ClsYoloGPU(param)

# 이미지 예제
img_path = '/content/yolov5/data/images/zidane.jpg'
if not os.path.exists(img_path):
    !wget https://ultralytics.com/images/zidane.jpg -O /content/yolov5/data/images/zidane.jpg
img = cv2.imread(img_path)
results = yolo.detectChar(img)
print(results)

__initCls = [Errno 2] No such file or directory: '/content/yolov5/yolov5/hubconf.py'
Using cuda device
NVIDIA A100-SXM4-40GB
0
1
detectChar = 'ClsYoloGPU' object has no attribute '_ClsYoloGPU__model'
[]


In [ ]:
# 데이터 증강 설정 파일 수정 (hyp.scratch-low.yaml 파일 수정 예시)
hyp_path = 'data/hyps/hyp.scratch-low.yaml'
!sed -i 's/mosaic: 1.0/mosaic: 1.5/' {hyp_path}
!sed -i 's/mixup: 0.5/mixup: 1.0/' {hyp_path}
!sed -i 's/scale: 0.5/scale: 0.9/' {hyp_path}

In [ ]:
# Roboflow에서 다운로드한 데이터셋 경로 확인
dataset_path = os.path.join(dataset.location, 'data.yaml')

In [10]:
# 학습 명령어 (전이 학습 및 적절한 배치 크기 사용)
!python train.py --data {dataset_path} --epochs 100 --weights yolov5s.pt --batch-size 16 --hyp {hyp_path} --img 640

2024-07-02 07:39:07.544948: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 07:39:07.545001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 07:39:07.546517: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/forklift-labeling-4/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi